In [3]:
from gnutools import fs
from bpd.dask import DataFrame
from bpd.dask import functions as F

In [4]:
# Define a function to be applied to each row
def word(f):
    return fs.name(fs.parent(f))

df = DataFrame({"filename": fs.listfiles("/tmp/GOOGLE/", [".wav"])})
df = df\
    .withColumn("name", F.apply(fs.name, F.col("filename")))\
    .withColumn("word", F.apply(word, F.col("filename")))\
    .reset_index()\
    .set_index("word")


In [5]:
# Define a function to be applied to a groupby
def first(L):
    return L[0]

dfg = df\
    .copy()\
    .select(["filename"])\
    .aggregate("word")\
    .withColumn("first", F.apply(first, F.col("filename")))\
    .drop_column(F.col("filename"))\
    .repartition(npartitions=8)

In [7]:
# Apply the join on df
df.join(dfg, on="word")

,index,filename,name,first
npartitions=8,,,,
,int64,object,object,object
,...,...,...,...
...,...,...,...,...
,...,...,...,...
,...,...,...,...


In [8]:
df.export("test.csv")

,index,filename,name,first
npartitions=8,,,,
,int64,object,object,object
,...,...,...,...
...,...,...,...,...
,...,...,...,...
,...,...,...,...
